# 结构化输出


响应格式
控制代理如何返回结构化数据：

- ProviderStrategy[StructuredResponseT] : 使用提供者原生的结构化输出
- ToolStrategy[StructuredResponseT] : 使用工具调用以获取结构化输出
- type[StructuredResponseT]: 模式类型 - 根据模型能力自动选择最佳策略
- None: 无结构化输出

## ProviderStrategy

In [ ]:
# Pydantic Schema

from pydantic import BaseModel
from langchain.agents import create_agent


class ContactInfo(BaseModel):
  """Contact information for a person."""
  name: str = Field(description="The name of the person")
  email: str = Field(description="The email address of the person")
  phone: str = Field(description="The phone number of the person")


agent = create_agent(
  model="gpt-5",
  tools=tools,
  response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
  "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [ ]:
# Dataclasses 数据类

from dataclasses import dataclass
from langchain.agents import create_agent


@dataclass
class ContactInfo:
  """Contact information for a person."""
  name: str  # The name of the person
  email: str  # The email address of the person
  phone: str  # The phone number of the person


agent = create_agent(
  model="gpt-5",
  tools=tools,
  response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
  "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [ ]:
# TypedDict

from typing_extensions import TypedDict
from langchain.agents import create_agent


class ContactInfo(TypedDict):
  """Contact information for a person."""
  name: str  # The name of the person
  email: str  # The email address of the person
  phone: str  # The phone number of the person


agent = create_agent(
  model="gpt-5",
  tools=tools,
  response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
  "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# {'name': 'John Doe', 'email': 'john@example.com', 'phone': '(555) 123-4567'}

In [ ]:
# JSON Schema

from langchain.agents import create_agent

contact_info_schema = {
  "type": "object",
  "description": "个人的联系信息",
  "properties": {
    "name": {"type": "string", "description": "姓名"},
    "email": {"type": "string", "description": "邮箱"},
    "phone": {"type": "string", "description": "手机号"}
  },
  "required": ["name", "email", "phone"]
}

agent = create_agent(
  model="gpt-5",
  tools=[],
  response_format=contact_info_schema  # Auto-selects ProviderStrategy
)

result = agent.invoke({
  "messages": [{"role": "user", "content": "提取信息: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# {'name': 'John Doe', 'email': 'john@example.com', 'phone': '(555) 123-4567'}

## 工具调用策略



In [ ]:
# pydantic

from pydantic import BaseModel, Field
from typing import Literal
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ProductReview(BaseModel):
  """Analysis of a product review."""
  rating: int | None = Field(description="The rating of the product", ge=1, le=5)
  sentiment: Literal["positive", "negative"] = Field(description="The sentiment of the review")
  key_points: list[str] = Field(description="The key points of the review. Lowercase, 1-3 words each.")


agent = create_agent(
  model="gpt-5",
  tools=tools,
  response_format=ToolStrategy(ProductReview)
)

result = agent.invoke({
  "messages": [
    {"role": "user", "content": "Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'"}]
})
result["structured_response"]
# ProductReview(rating=5, sentiment='positive', key_points=['fast shipping', 'expensive'])

In [ ]:
# dataclass

from dataclasses import dataclass
from typing import Literal
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


@dataclass
class ProductReview:
  """Analysis of a product review."""
  rating: int | None  # The rating of the product (1-5)
  sentiment: Literal["positive", "negative"]  # The sentiment of the review
  key_points: list[str]  # The key points of the review


agent = create_agent(
  model="gpt-5",
  tools=tools,
  response_format=ToolStrategy(ProductReview)
)

result = agent.invoke({
  "messages": [
    {"role": "user", "content": "Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'"}]
})
result["structured_response"]
# ProductReview(rating=5, sentiment='positive', key_points=['fast shipping', 'expensive'])

In [ ]:
# TypedDict

from typing import Literal
from typing_extensions import TypedDict
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ProductReview(TypedDict):
  """Analysis of a product review."""
  rating: int | None  # The rating of the product (1-5)
  sentiment: Literal["positive", "negative"]  # The sentiment of the review
  key_points: list[str]  # The key points of the review


agent = create_agent(
  model="gpt-5",
  tools=tools,
  response_format=ToolStrategy(ProductReview)
)

result = agent.invoke({
  "messages": [
    {"role": "user", "content": "Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'"}]
})
result["structured_response"]
# {'rating': 5, 'sentiment': 'positive', 'key_points': ['fast shipping', 'expensive']}

In [ ]:
# JSON Schema

from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

product_review_schema = {
  "type": "object",
  "description": "Analysis of a product review.",
  "properties": {
    "rating": {
      "type": ["integer", "null"],
      "description": "The rating of the product (1-5)",
      "minimum": 1,
      "maximum": 5
    },
    "sentiment": {
      "type": "string",
      "enum": ["positive", "negative"],
      "description": "The sentiment of the review"
    },
    "key_points": {
      "type": "array",
      "items": {"type": "string"},
      "description": "The key points of the review"
    }
  },
  "required": ["sentiment", "key_points"]
}

agent = create_agent(
  model="gpt-5",
  tools=tools,
  response_format=ToolStrategy(product_review_schema)
)

result = agent.invoke({
  "messages": [
    {"role": "user", "content": "Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'"}]
})
result["structured_response"]
# {'rating': 5, 'sentiment': 'positive', 'key_points': ['fast shipping', 'expensive']}

In [ ]:
# Union types

from pydantic import BaseModel, Field
from typing import Literal, Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ProductReview(BaseModel):
  """Analysis of a product review."""
  rating: int | None = Field(description="The rating of the product", ge=1, le=5)
  sentiment: Literal["positive", "negative"] = Field(description="The sentiment of the review")
  key_points: list[str] = Field(description="The key points of the review. Lowercase, 1-3 words each.")


class CustomerComplaint(BaseModel):
  """A customer complaint about a product or service."""
  issue_type: Literal["product", "service", "shipping", "billing"] = Field(description="The type of issue")
  severity: Literal["low", "medium", "high"] = Field(description="The severity of the complaint")
  description: str = Field(description="Brief description of the complaint")


agent = create_agent(
  model="gpt-5",
  tools=tools,
  response_format=ToolStrategy(Union[ProductReview, CustomerComplaint])
)

result = agent.invoke({
  "messages": [
    {"role": "user", "content": "Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'"}]
})
result["structured_response"]
# ProductReview(rating=5, sentiment='positive', key_points=['fast shipping', 'expensive'])ß

### 自定义工具消息内容

默认情况下，当你用 ToolStrategy 让模型返回结构化输出时，LangChain 会自动往消息流里添加一条 ToolMessage，它的默认内容是：

Returning structured response: {...}


但很多时候，你不希望用户或者下游逻辑看到这种“生硬的系统提示”，就需要用 tool_message_content 来改写。

In [10]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class MeetingAction(BaseModel):
  """从会议记录中提取的行动事项。"""
  task: str = Field(description="要完成的具体任务")
  assignee: str = Field(description="负责这项任务的人")
  priority: Literal["low", "medium", "high"] = Field(description="任务的优先级")


agent = create_agent(
  model="gpt-5",
  tools=[],
  response_format=ToolStrategy(
    schema=MeetingAction,
    tool_message_content="行动事项已记录并添加到会议纪要中！"
  )
)

response = agent.invoke({
  "messages": [{"role": "user", "content": "根据我们的会议内容：Sarah 需要尽快更新项目时间表。"}]
})


### 错误处理

In [13]:
from pydantic import BaseModel, Field
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ContactInfo(BaseModel):
  name: str = Field(description="姓名")
  email: str = Field(description="邮箱地址")


class EventDetails(BaseModel):
  event_name: str = Field(description="事件名称")
  date: str = Field(description="事件日期")


agent = create_agent(
  model="gpt-5",
  tools=[],
  system_prompt="你是一个信息提取助手，可以从文本中提取联系信息或事件详情。",
  response_format=ToolStrategy(Union[ContactInfo, EventDetails])  # Default: handle_errors=True
)

agent.invoke({
  "messages": [{"role": "user", "content": "信息摘要：John Doe（john@email.com）将于3月15日组织一场技术大会。"}]
})

{'messages': [HumanMessage(content='信息摘要：John Doe（john@email.com）将于3月15日组织一场技术大会。', additional_kwargs={}, response_metadata={}, id='65bdf6e0-29c0-4e9e-9075-da50dc10e9c1'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 646, 'prompt_tokens': 202, 'total_tokens': 848, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CXpyr4wbQAAEpxVVY6jGtxHuI9LpJ', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--e90a513d-562c-4c18-9186-5329388e543c-0', tool_calls=[{'name': 'ContactInfo', 'args': {'email': 'john@email.com', 'name': 'John Doe'}, 'id': 'call_1LtUaGS69ueAT5Yi3UtrIWe5', 'type': 'tool_call'}, {'name': 'EventDetails', 'args': {'date': '3月15日', 'event_na

### 模式验证错误

In [14]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ProductRating(BaseModel):
  rating: int | None = Field(description="Rating from 1-5", ge=1, le=5)
  comment: str = Field(description="Review comment")


agent = create_agent(
  model="gpt-5",
  tools=[],
  response_format=ToolStrategy(
    schema=ProductRating,
    handle_errors="Please provide a valid rating between 1-5 and include a comment."
  ),
  system_prompt="You are a helpful assistant that parses product reviews. Do not make any field or value up."
)

agent.invoke({
  "messages": [{"role": "user", "content": "Parse this: Amazing product, 10/10!"}]
})

{'messages': [HumanMessage(content='Parse this: Amazing product, 10/10!', additional_kwargs={}, response_metadata={}, id='2aa0bd9d-d93b-449c-a160-58f6dbf36480'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 675, 'prompt_tokens': 180, 'total_tokens': 855, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 640, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CXq3SrgGW1gqLK9qGutCgNQ5mAp6X', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--99a1a6d1-34fd-4206-a6d3-e7b798d6909c-0', tool_calls=[{'name': 'ProductRating', 'args': {'comment': 'Amazing product, 10/10!', 'rating': None}, 'id': 'call_kd9YvSzEGV3TXRzcZoFPnk0Q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 180, 'output_tokens': 675, '